# Import packages

In [1]:
# LINC API
import hackathon_linc as lh
lh.init('92438482-5598-4e17-8b34-abe17aa8f598')

# For data handling
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta



    Welcome to the LINC Hackathon! Your token is now saved in the Console. 
    That means you don't need to carry that out when using the other functions
    as long as you don't close your console. 
    
    This function is only to be used once to authenticate your token.
    
    Happy coding!

    


# Load data


In [2]:
# Read historical data
df = pd.read_csv('historical_data/stockPrices_hourly.csv')

# Convert date column to datetime objects
df['gmtTime'] = pd.to_datetime(df['gmtTime'])

# Add price column
df['price'] = (df['askMedian'] + df['bidMedian']) / 2

df.head()

,gmtTime,askMedian,bidMedian,askVolume,bidVolume,spreadMedian,symbol,price
0,2015-04-22 07:00:00,31.563367,31.549133,340171.136213,238282.318260,0.014100,STOCK5,31.556250
1,2015-04-22 07:00:00,92.565750,92.520333,77727.916568,66743.550157,0.045450,STOCK2,92.543042
2,2015-04-22 07:00:00,112.642667,112.560167,124831.366768,140258.649178,0.082542,STOCK3,112.601417
3,2015-04-22 07:00:00,116.097050,115.896233,5192.116635,7605.816630,0.200833,STOCK4,115.996642
4,2015-04-22 07:00:00,11.846933,11.836850,529883.184283,330190.685393,0.010050,STOCK7,11.841892


# Calculate and plot hourly RSI

In [10]:
from functions.rsi import calculate_daily_rsi, calculate_hourly_rsi
import holoviews as hv
import hvplot.pandas
hvplot.extension('bokeh')

# Get random symbol or choose symbol
symbol = df['symbol'].drop_duplicates().sample(n=1).values[0]
# symbol = 'STOCK5'

# Calculate RSI
periods = 8*14
df_rsi = calculate_hourly_rsi(df[df['symbol'] == symbol], periods)

# RSI lines
upper_line = 70
lower_line = 30
left_x = df_rsi['gmtTime'].min()  # Leftmost x-value

df_rsi.hvplot.line(
    x='gmtTime',
    y='RSI',
    by='symbol',
    height=600,
    width=1000,
    legend='right',
    xlabel='Greenwich Mean Time (Date only)',
    ylabel='RSI',
    title=f'Hourly RSI with period {periods} for stock {symbol}'
    ) * \
    hv.HLine(y=upper_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=upper_line, text=f'RSI{upper_line}').opts(text_align='left', text_color='black', text_font_size='20pt') * \
    hv.HLine(y=lower_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=lower_line, text=f'RSI{lower_line}').opts(text_align='left', text_color='black', text_font_size='20pt')


:Overlay
   .NdOverlay.I :NdOverlay   [symbol]
      :Curve   [gmtTime]   (RSI)
   .HLine.I     :HLine   [x,y]
   .Text.I      :Text   [x,y]
   .HLine.II    :HLine   [x,y]
   .Text.II     :Text   [x,y]

# Calculate and plot daily RSI

In [11]:
from functions.rsi import calculate_daily_rsi, calculate_hourly_rsi
import holoviews as hv
import hvplot.pandas
hvplot.extension('bokeh')

# Get random symbol or choose symbol
symbol = df['symbol'].drop_duplicates().sample(n=1).values[0]
# symbol = 'STOCK5'

# Calculate RSI
periods = 14
df_rsi = calculate_daily_rsi(df[df['symbol'] == symbol], periods)

# RSI lines
upper_line = 70
lower_line = 30
left_x = df_rsi['date'].min()  # Leftmost x-value

df_rsi.hvplot.line(
    x='date',
    y='RSI',
    by='symbol',
    height=600,
    width=1000,
    legend='right',
    xlabel='Greenwich Mean Time (Date only)',
    ylabel='RSI',
    title=f'Daily RSI with period {periods} for stock {symbol}'
    ) * \
    hv.HLine(y=upper_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=upper_line, text=f'RSI{upper_line}').opts(text_align='left', text_color='black', text_font_size='20pt') * \
    hv.HLine(y=lower_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=lower_line, text=f'RSI{lower_line}').opts(text_align='left', text_color='black', text_font_size='20pt')


:Overlay
   .NdOverlay.I :NdOverlay   [symbol]
      :Curve   [date]   (RSI)
   .HLine.I     :HLine   [x,y]
   .Text.I      :Text   [x,y]
   .HLine.II    :HLine   [x,y]
   .Text.II     :Text   [x,y]

# Mdified Markowitz portfolio optimization problem

$$
\max{\mathbf{w}} \left( \mathbb{E}[R_p] - \lambda \cdot VaR_\alpha (R_p) \right)
$$

$$
\text{Subject to:}
$$

$$
\sum_{i=1}^{n} w_i = 1, \quad w_i \geq 0 \quad \forall i
$$

$$
\mathbb{E}[R_p] = \mathbf{w}^\top \mathbf{\mu}, \\
VaR_\alpha (R_p) = - \left( \mathbf{w}^\top \mathbf{\mu} + \lambda \cdot z_\alpha \cdot \sqrt{\mathbf{w}^\top \Sigma \mathbf{w}} \right) \implies
$$

$$
\max_{\mathbf{w}} \left( (1 + \lambda) \mathbf{w}^\top \mathbf{\mu} + \lambda \cdot z_\alpha \cdot \sqrt{\mathbf{w}^\top \Sigma \mathbf{w}} \right)
$$

$$
\text{Subject to:}
$$

$$
\sum_{i=1}^{n} w_i = 1, \quad w_i \geq 0 \quad \forall i
$$

$$
\text{Notation:}
$$
$$
\begin{aligned}
&\mathbf{w} && \text{: Vector of portfolio weights} \\
&\mathbf{\mu} && \text{: Vector of expected returns for each asset} \\
&\Sigma && \text{: Covariance matrix of asset returns} \\
&\mathbb{E}[R_p] && \text{: Expected portfolio return, } \mathbf{w}^\top \mathbf{\mu} \\
&\sigma_p && \text{: Portfolio standard deviation, } \sqrt{\mathbf{w}^\top \Sigma \mathbf{w}} \\
&z_\alpha && \text{: Z-score for confidence level } \alpha \text{ (e.g., 1.645 for 95\% confidence)} \\
&\text{VaR}_\alpha(R_p) && \text{: Value at Risk at confidence level } \alpha \text{ (risk measure)} \\
&\lambda && \text{: Risk-aversion parameter (controls trade-off between return and risk)}
\end{aligned}
$$


## Calculate $\mu$ and $\Sigma$

In [63]:
# Calculate expected return per stock
df_return = df.groupby('symbol').agg(
    first_price=('price', 'first'),
    last_price=('price', 'last')
).reset_index()

# Assuming we know the number of years
years = 5  

df_return['returns'] = np.array((df_return['last_price'] / df_return['first_price']) ** (1 / years) - 1)
expected_returns = df_return['returns']

# Calculate daily returns per stock
df['returns'] = df.groupby('symbol')['price'].pct_change()

# Compute expected returns (mean return for each asset)
# expected_returns = df.groupby('symbol')['returns'].mean()

# Pivot the dataframe so that each column represents a stock's returns
returns_matrix = df.pivot(index='gmtTime', columns='symbol', values='returns')

# Compute the covariance matrix
cov_matrix = returns_matrix.cov()



## Optimize portfolio

In [64]:
from scipy.optimize import minimize

lambda_risk = 0.9   # Risk-aversion parameter (tune this)
z_alpha = 1.645     # Z-score for 95% confidence level
num_assets = len(expected_returns)

# Objective function: Maximize (expected return - risk-adjusted VaR)
def objective(w):
    w = np.array(w)
    portfolio_return = np.dot(w, expected_returns)  # w^T * mu
    portfolio_std = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))  # sqrt(w^T * Sigma * w)
    var_risk = - (portfolio_return + z_alpha * portfolio_std)  # VaR term
    return -((1 + lambda_risk) * portfolio_return + lambda_risk * var_risk)  # Negative for minimization

# Constraints
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Sum of weights = 1
]

# Bounds (no short-selling: weights between 0 and 1)
bounds = [(0, 1) for _ in range(num_assets)]

# Initial guess (equal allocation)
w0 = np.ones(num_assets) / num_assets

# Solve optimization problem
result = minimize(objective, w0, bounds=bounds, constraints=constraints)#, method='SLSQP')

# Optimal portfolio weights
optimal_weights = result.x

# Optimal portfolio returns
optimal_portfolio_return = np.dot(optimal_weights, expected_returns)

# Optimal Value at Risk
optimal_portfolio_std = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))  # sqrt(w^T * Sigma * w)
optimal_var_risk = - (optimal_portfolio_return + z_alpha * optimal_portfolio_std)  # VaR term

print("Optimal Portfolio Weights:", optimal_weights)
print("Optimal Portfolio Return:", optimal_portfolio_return)
print("Optimal Portfolio VaR:", optimal_var_risk)



Optimal Portfolio Weights: [1.00000000e+00 0.00000000e+00 1.59594560e-16 0.00000000e+00
 3.69496100e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 9.71445147e-17]
Optimal Portfolio Return: 0.22737867445643886
Optimal Portfolio VaR: -0.23563572349778097


In [65]:
z_alpha = 1.645  # Z-score for 95% confidence level
num_assets = len(expected_returns)

# Function to compute optimal portfolio return for a given lambda
def compute_optimal_return(lambda_risk):
    def objective(w):
        portfolio_return = np.dot(w, expected_returns)  # w^T * mu
        portfolio_std = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))  # sqrt(w^T * Sigma * w)
        var_risk = - (portfolio_return + z_alpha * portfolio_std)  # VaR term
        return -((1 + lambda_risk) * portfolio_return + lambda_risk * var_risk)  # Negative for minimization

    # Constraints
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]  # Sum of weights = 1

    # Bounds (no short-selling)
    bounds = [(0, 1) for _ in range(num_assets)]

    # Initial guess (equal allocation)
    w0 = np.ones(num_assets) / num_assets

    # Solve optimization
    result = minimize(objective, w0, bounds=bounds, constraints=constraints, method='SLSQP')
    optimal_weights = result.x

    # Compute expected portfolio return for these weights
    optimal_return = np.dot(optimal_weights, expected_returns)
    
    return optimal_return

# Loop over different values of lambda
lambda_values = np.linspace(0, 2, 20)  # Vary lambda from 0 to 2 in 20 steps
expected_returns_list = [compute_optimal_return(l) for l in lambda_values]

# Create DataFrame for plotting
df_plot = pd.DataFrame({'lambda': lambda_values, 'expected_return': expected_returns_list})

# Plot using hvplot
df_plot.hvplot.line(
    x='lambda', 
    y='expected_return', 
    title="Expected Return vs. Risk Aversion (λ)",
    xlabel="Risk Aversion (λ)",
    ylabel="Expected Return (𝐄[Rₚ])"
    )


:Curve   [lambda]   (expected_return)